# Predicting Dengue in San Juan and Iquitos
**by  Mahir Dalal | Division B**

In [1]:
# Setup
import pandas as pd
import numpy as np

import seaborn as sns # for visualiation
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor

# Importing and Preparing the Training Data

In [2]:
training_features_df = pd.read_csv("data/dengue_features_train.csv")
training_features_df.head(3)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.12260,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.16990,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.03225,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4


In [3]:
training_labels_df = pd.read_csv("data/dengue_labels_train.csv")
training_labels_df.head(3)

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4


In [4]:
training_df = pd.merge(training_features_df,training_labels_df, how = "left", on = ["city","year","weekofyear"])
training_df.head(3)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.12260,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.16990,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.03225,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4


In [5]:
print("Training Data has", len(training_df),"rows")

Training Data has 1456 rows


## Adding Time Columns to the Training Data

In [6]:
# Converting our week start date column to Date Time Type
training_df['week_start_date'] = pd.to_datetime(training_df['week_start_date'])

# Confirming our column types have updated
training_df.dtypes

city                                             object
year                                              int64
weekofyear                                        int64
week_start_date                          datetime64[ns]
ndvi_ne                                         float64
ndvi_nw                                         float64
ndvi_se                                         float64
ndvi_sw                                         float64
precipitation_amt_mm                            float64
reanalysis_air_temp_k                           float64
reanalysis_avg_temp_k                           float64
reanalysis_dew_point_temp_k                     float64
reanalysis_max_air_temp_k                       float64
reanalysis_min_air_temp_k                       float64
reanalysis_precip_amt_kg_per_m2                 float64
reanalysis_relative_humidity_percent            float64
reanalysis_sat_precip_amt_mm                    float64
reanalysis_specific_humidity_g_per_kg           

In [7]:
# New Column, for the Month Number
training_df["month"] = training_df["week_start_date"].dt.month

# New Column, for the Day of Month Number
training_df["dayofmonth"] = training_df["week_start_date"].dt.day

# New Column, for the Day of Year
training_df["dayofyear"] = training_df["week_start_date"].dt.dayofyear

# New Column, for telling us whether the year is an odd year or an even year
training_df.loc[(training_df["year"]%2 == 1), "odd_year"] = 1 # yes  
training_df.loc[(training_df["year"]%2 == 0), "odd_year"] = 0 # no

training_df.head(3)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,month,dayofmonth,dayofyear,odd_year
0,sj,1990,18,1990-04-30,0.12260,0.103725,0.198483,0.177617,12.42,297.572857,...,25.442857,6.900000,29.4,20.0,16.0,4,4,30,120,0.0
1,sj,1990,19,1990-05-07,0.16990,0.142175,0.162357,0.155486,22.82,298.211429,...,26.714286,6.371429,31.7,22.2,8.6,5,5,7,127,0.0
2,sj,1990,20,1990-05-14,0.03225,0.172967,0.157200,0.170843,34.54,298.781429,...,26.714286,6.485714,32.2,22.8,41.4,4,5,14,134,0.0


# San Juan Machine Learning Model

## Preparing the Data

In [8]:
sj_train_df = training_df.loc[training_df["city"] == "sj"].reset_index()
sj_train_df.head(3)

,index,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,month,dayofmonth,dayofyear,odd_year
0,0,sj,1990,18,1990-04-30,0.12260,0.103725,0.198483,0.177617,12.42,...,25.442857,6.900000,29.4,20.0,16.0,4,4,30,120,0.0
1,1,sj,1990,19,1990-05-07,0.16990,0.142175,0.162357,0.155486,22.82,...,26.714286,6.371429,31.7,22.2,8.6,5,5,7,127,0.0
2,2,sj,1990,20,1990-05-14,0.03225,0.172967,0.157200,0.170843,34.54,...,26.714286,6.485714,32.2,22.8,41.4,4,5,14,134,0.0


## Handling the Missing Values

In [9]:
# Calculating the Total Missing Values for San Juan
sj_train_df.isnull().sum()

index                                      0
city                                       0
year                                       0
weekofyear                                 0
week_start_date                            0
ndvi_ne                                  191
ndvi_nw                                   49
ndvi_se                                   19
ndvi_sw                                   19
precipitation_amt_mm                       9
reanalysis_air_temp_k                      6
reanalysis_avg_temp_k                      6
reanalysis_dew_point_temp_k                6
reanalysis_max_air_temp_k                  6
reanalysis_min_air_temp_k                  6
reanalysis_precip_amt_kg_per_m2            6
reanalysis_relative_humidity_percent       6
reanalysis_sat_precip_amt_mm               9
reanalysis_specific_humidity_g_per_kg      6
reanalysis_tdtr_k                          6
station_avg_temp_c                         6
station_diur_temp_rng_c                    6
station_ma

In [10]:
# Forward Filling Our Missing Values
sj_train_df = sj_train_df.fillna(method = 'ffill') 

# Calculating the Total Missing Values for San Juan
sj_train_df.isnull().sum()

index                                    0
city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min

## Making the Average Vegetation Index

In [11]:
sj_train_df["ndvi_avg"] = (sj_train_df["ndvi_ne"] 
                            + sj_train_df["ndvi_se"]  
                            + sj_train_df["ndvi_nw"]
                            + sj_train_df["ndvi_nw"])/4
sj_train_df.head()

,index,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,month,dayofmonth,dayofyear,odd_year,ndvi_avg
0,0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,...,6.900000,29.4,20.0,16.0,4,4,30,120,0.0,0.132133
1,1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,...,6.371429,31.7,22.2,8.6,5,5,7,127,0.0,0.154152
2,2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,...,6.485714,32.2,22.8,41.4,4,5,14,134,0.0,0.133846
3,3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,...,6.771429,33.3,23.3,4.0,3,5,21,141,0.0,0.211581
4,4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,...,9.371429,35.0,23.9,5.8,6,5,28,148,0.0,0.242950


# Understanding Normalization

In [28]:
data=[[-1,2],[-0.5,6],[0,10],[1,18]]
data

[[-1, 2], [-0.5, 6], [0, 10], [1, 18]]

In [29]:
scaler=MinMaxScaler()

In [30]:
scaler.fit(data)

MinMaxScaler()

In [31]:
scaler.transform(data)

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

# Normalising our Columns

In [32]:
sj_train_df.columns

Index(['index', 'city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne',
       'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
       'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
       'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
       'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'total_cases', 'month',
       'dayofmonth', 'dayofyear', 'odd_year', 'ndvi_avg'],
      dtype='object')

In [33]:
sj_train_df_n=MinMaxScaler().fit_transform(sj_train_df[sj_train_df.columns[5:25]])
sj_train_df_n

array([[0.58783972, 0.62676332, 0.52370092, ..., 0.30337079, 0.28205128,
        0.05230467],
       [0.64041572, 0.66981079, 0.43529974, ..., 0.56179775, 0.56410256,
        0.02811376],
       [0.48741177, 0.70428426, 0.42268026, ..., 0.61797753, 0.64102564,
        0.13533835],
       ...,
       [0.27905297, 0.45157859, 0.37510328, ..., 0.30337079, 0.5       ,
        0.10035959],
       [0.45356528, 0.45157859, 0.53695564, ..., 0.30337079, 0.5       ,
        0.03661327],
       [0.41043739, 0.4990297 , 0.22719909, ..., 0.56179775, 0.70512821,
        0.00098071]])

In [34]:
sj_train_df_n=pd.DataFrame(sj_train_df_n,columns=sj_train_df.columns[5:25],index=sj_train_df.index)
sj_train_df_n

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,0.587840,0.626763,0.523701,0.541889,0.031797,0.261008,0.269185,0.339933,0.307692,0.452055,0.056091,0.318138,0.031797,0.297392,0.413953,0.359684,0.440318,0.303371,0.282051,0.052305
1,0.640416,0.669811,0.435300,0.492142,0.058423,0.362993,0.384888,0.528474,0.476923,0.520548,0.031446,0.510214,0.058423,0.473460,0.330233,0.535573,0.342175,0.561798,0.564103,0.028114
2,0.487412,0.704284,0.422680,0.526662,0.088428,0.454027,0.456907,0.710356,0.415385,0.643836,0.045749,0.734988,0.088428,0.664509,0.306977,0.535573,0.363395,0.617978,0.641026,0.135338
3,0.594546,0.785005,0.594845,0.672866,0.039324,0.486881,0.514758,0.695111,0.553846,0.602740,0.024365,0.652660,0.039324,0.641761,0.348837,0.640316,0.416446,0.741573,0.705128,0.013076
4,0.669649,0.804187,0.652699,0.698613,0.019252,0.571755,0.586777,0.757841,0.630769,0.671233,0.021385,0.658555,0.019252,0.711300,0.539535,0.843874,0.899204,0.932584,0.782051,0.018960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,0.538098,0.465965,0.797737,0.808538,0.069611,0.162902,0.139315,0.300859,0.292308,0.246575,0.013234,0.360433,0.069611,0.249676,0.497674,0.304348,0.233422,0.370787,0.423077,0.005884
932,0.409326,0.491790,0.330113,0.291862,0.009780,0.342231,0.349469,0.440687,0.307692,0.534247,0.006433,0.377365,0.009780,0.381542,0.441860,0.470356,0.424403,0.438202,0.564103,0.001635
933,0.279053,0.451579,0.375103,0.460063,0.043420,0.242984,0.239669,0.379008,0.246154,0.438356,0.061350,0.397861,0.043420,0.319586,0.269767,0.298419,0.220159,0.303371,0.500000,0.100360
934,0.453565,0.451579,0.536956,0.614327,0.000000,0.270135,0.275089,0.322762,0.292308,0.452055,0.008449,0.266315,0.000000,0.277418,0.465116,0.341897,0.318302,0.303371,0.500000,0.036613


In [36]:
# adding time data to normalized data
sj_train_df_n["year"]=sj_train_df["year"]
sj_train_df_n["month"]=sj_train_df["month"]
sj_train_df_n["weekofyear"]=sj_train_df["weekofyear"]
sj_train_df_n["odd_year"]=sj_train_df["odd_year"]
sj_train_df_n["dayofmonth"]=sj_train_df["dayofmonth"]
sj_train_df_n["dayofyear"]=sj_train_df["dayofyear"]
sj_train_df_n

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,year,month,weekofyear,odd_year,dayofmonth,dayofyear
0,0.587840,0.626763,0.523701,0.541889,0.031797,0.261008,0.269185,0.339933,0.307692,0.452055,...,0.440318,0.303371,0.282051,0.052305,1990,4,18,0.0,30,120
1,0.640416,0.669811,0.435300,0.492142,0.058423,0.362993,0.384888,0.528474,0.476923,0.520548,...,0.342175,0.561798,0.564103,0.028114,1990,5,19,0.0,7,127
2,0.487412,0.704284,0.422680,0.526662,0.088428,0.454027,0.456907,0.710356,0.415385,0.643836,...,0.363395,0.617978,0.641026,0.135338,1990,5,20,0.0,14,134
3,0.594546,0.785005,0.594845,0.672866,0.039324,0.486881,0.514758,0.695111,0.553846,0.602740,...,0.416446,0.741573,0.705128,0.013076,1990,5,21,0.0,21,141
4,0.669649,0.804187,0.652699,0.698613,0.019252,0.571755,0.586777,0.757841,0.630769,0.671233,...,0.899204,0.932584,0.782051,0.018960,1990,5,22,0.0,28,148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,0.538098,0.465965,0.797737,0.808538,0.069611,0.162902,0.139315,0.300859,0.292308,0.246575,...,0.233422,0.370787,0.423077,0.005884,2008,3,13,0.0,25,85
932,0.409326,0.491790,0.330113,0.291862,0.009780,0.342231,0.349469,0.440687,0.307692,0.534247,...,0.424403,0.438202,0.564103,0.001635,2008,4,14,0.0,1,92
933,0.279053,0.451579,0.375103,0.460063,0.043420,0.242984,0.239669,0.379008,0.246154,0.438356,...,0.220159,0.303371,0.500000,0.100360,2008,4,15,0.0,8,99
934,0.453565,0.451579,0.536956,0.614327,0.000000,0.270135,0.275089,0.322762,0.292308,0.452055,...,0.318302,0.303371,0.500000,0.036613,2008,4,16,0.0,15,106


# Splitting our Data

In [37]:
train_features_sj,test_features_sj, train_outcome_sj, test_outcome_sj=train_test_split(sj_train_df_n,sj_train_df["total_cases"],test_size=0.3)

# Building our Machine Learning Model

In [40]:
# Fit the model to our (training) data, and use it to predict on our "test data"
model=KNeighborsRegressor(n_neighbors=12, weights="distance")
model_predictions=model.fit(train_features_sj,train_outcome_sj).predict(test_features_sj)

# Calculating the mean absolute error
mean_absolute_error_of_model=mean_absolute_error(test_outcome_sj,model_predictions)
mean_absolute_error_of_model

24.219119058634384

# Identifying the most important columns

In [48]:
model=ExtraTreesRegressor()
model.fit(train_features_sj,train_outcome_sj)
feature_importance_df=pd.DataFrame({"feature_name":train_features_sj.columns,"importance":model.feature_importances_})
feature_importance_df.sort_values(["importance"],ascending=False)

,feature_name,importance
20,year,0.172722
2,ndvi_se,0.108428
3,ndvi_sw,0.092380
23,odd_year,0.086971
21,month,0.070527
22,weekofyear,0.063258
25,dayofyear,0.059000
7,reanalysis_dew_point_temp_k,0.042661
13,reanalysis_specific_humidity_g_per_kg,0.041303
17,station_max_temp_c,0.029654


# Iquitos Machine Learning Model

## Preparing the Data

In [12]:
iq_train_df = training_df.loc[training_df["city"] == "iq"].reset_index()
iq_train_df.head(3)

,index,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,month,dayofmonth,dayofyear,odd_year
0,936,iq,2000,26,2000-07-01,0.192886,0.132257,0.340886,0.247200,25.41,...,26.4,10.775000,32.5,20.7,3.0,0,7,1,183,0.0
1,937,iq,2000,27,2000-07-08,0.216833,0.276100,0.289457,0.241657,60.61,...,26.9,11.566667,34.0,20.8,55.6,0,7,8,190,0.0
2,938,iq,2000,28,2000-07-15,0.176757,0.173129,0.204114,0.128014,55.52,...,26.8,11.466667,33.0,20.7,38.1,0,7,15,197,0.0


## Handling the Missing Values

In [13]:
# Seeing our missing values
iq_train_df.isnull().sum()

index                                     0
city                                      0
year                                      0
weekofyear                                0
week_start_date                           0
ndvi_ne                                   3
ndvi_nw                                   3
ndvi_se                                   3
ndvi_sw                                   3
precipitation_amt_mm                      4
reanalysis_air_temp_k                     4
reanalysis_avg_temp_k                     4
reanalysis_dew_point_temp_k               4
reanalysis_max_air_temp_k                 4
reanalysis_min_air_temp_k                 4
reanalysis_precip_amt_kg_per_m2           4
reanalysis_relative_humidity_percent      4
reanalysis_sat_precip_amt_mm              4
reanalysis_specific_humidity_g_per_kg     4
reanalysis_tdtr_k                         4
station_avg_temp_c                       37
station_diur_temp_rng_c                  37
station_max_temp_c              

In [14]:
# Forward Filling Our Missing Values
iq_train_df = iq_train_df.fillna(method = 'ffill') 

# Calculating the Total Missing Values for San Juan
iq_train_df.isnull().sum()

index                                    0
city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min

## Making the Average Vegetation Index

In [15]:
iq_train_df["ndvi_avg"] = (iq_train_df["ndvi_ne"] 
                            + iq_train_df["ndvi_se"]  
                            + iq_train_df["ndvi_nw"]
                            + iq_train_df["ndvi_nw"])/4
iq_train_df.head()

,index,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,month,dayofmonth,dayofyear,odd_year,ndvi_avg
0,936,iq,2000,26,2000-07-01,0.192886,0.132257,0.340886,0.247200,25.41,...,10.775000,32.5,20.7,3.0,0,7,1,183,0.0,0.199571
1,937,iq,2000,27,2000-07-08,0.216833,0.276100,0.289457,0.241657,60.61,...,11.566667,34.0,20.8,55.6,0,7,8,190,0.0,0.264623
2,938,iq,2000,28,2000-07-15,0.176757,0.173129,0.204114,0.128014,55.52,...,11.466667,33.0,20.7,38.1,0,7,15,197,0.0,0.181782
3,939,iq,2000,29,2000-07-22,0.227729,0.145429,0.254200,0.200314,5.60,...,10.533333,31.5,14.7,30.0,0,7,22,204,0.0,0.193196
4,940,iq,2000,30,2000-07-29,0.328643,0.322129,0.254371,0.361043,62.76,...,11.480000,33.3,19.1,4.0,0,7,29,211,0.0,0.306818
